In [1]:
import os
import numpy as np

import pandas as pd

import tensorflow as tf
import tensorflow_probability as tfp

import matplotlib.pyplot as plt
from tabulate import tabulate

2024-08-20 17:20:23.781066: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-20 17:20:26.139969: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-20 17:20:26.140007: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-20 17:20:26.146720: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-20 17:20:26.898571: I tensorflow/core/platform/cpu_feature_g

# New York data: with structural zeros

Load the clean dataframe

In [2]:
df = pd.read_csv("data/NY_dataframe.csv")

In [3]:
df.head()

,OWNERSHP,MORTGAGE,AGE,SEX,MARST,RACESING,EDUC,EMPSTAT,DISABWRK,VETSTAT
0,1,1,6,1,2,0,9,1,1,2
1,1,1,4,1,5,0,5,3,1,1
2,1,1,3,1,5,0,4,3,1,1
3,1,2,7,0,0,0,9,1,1,2
4,1,2,7,1,0,0,8,1,1,1


In [4]:
set_row = []
for row in df.columns:
	set_row.append(row)
	for col in df.columns:
		if row!=col:
			if (col in set_row)==False:
				tab = pd.crosstab( index=df[row], columns=df[col])
				print(tab.to_latex())

\begin{tabular}{lrrrr}
\toprule
MORTGAGE & 0 & 1 & 2 & 3 \\
OWNERSHP &  &  &  &  \\
\midrule
0 & 29039 & 0 & 0 & 0 \\
1 & 0 & 162251 & 406049 & 3622 \\
2 & 352115 & 0 & 0 & 0 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrrrr}
\toprule
AGE & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 \\
OWNERSHP &  &  &  &  &  &  &  &  &  \\
\midrule
0 & 734 & 116 & 219 & 224 & 9584 & 3881 & 4438 & 2762 & 7081 \\
1 & 117956 & 8604 & 8423 & 8622 & 34220 & 67427 & 142876 & 128297 & 55497 \\
2 & 82372 & 4572 & 4451 & 4609 & 38736 & 73798 & 72579 & 46603 & 24395 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrr}
\toprule
SEX & 0 & 1 \\
OWNERSHP &  &  \\
\midrule
0 & 16233 & 12806 \\
1 & 277865 & 294057 \\
2 & 165171 & 186944 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrr}
\toprule
MARST & 0 & 1 & 2 & 3 & 4 & 5 \\
OWNERSHP &  &  &  &  &  &  \\
\midrule
0 & 0 & 10157 & 765 & 991 & 3289 & 13837 \\
1 & 271998 & 6867 & 7899 & 30491 & 32599 & 222068 \\
2 & 89146 & 10723 & 14335 & 27881 & 19246 & 190784 \\
\b

Build the domain vectors

In [5]:
n_j = np.zeros(len(df.columns))
for index in range(len(df.columns)):

    n_j[index] = df[df.columns[index]].max()
    
n_j = tf.convert_to_tensor(n_j+1, dtype = tf.float32)
max_categ = tf.expand_dims(tf.cast(tf.linspace(0, int(tf.reduce_max(n_j)) - 1, int(tf.reduce_max(n_j))), dtype = tf.float32), axis = 0)*tf.ones((tf.shape(n_j)[0], int(tf.reduce_max(n_j))))
ones_condition = max_categ<tf.expand_dims(n_j, axis = -1)*tf.ones((tf.shape(n_j)[0], int(tf.reduce_max(n_j))))

one_n_j = tf.where(ones_condition, tf.ones(tf.shape(ones_condition)), tf.zeros(tf.shape(ones_condition)))

2024-08-20 17:20:36.746065: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-20 17:20:38.216274: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-20 17:20:38.216321: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-20 17:20:38.235076: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-20 17:20:38.235127: I tensorflow/compile

Number of represented cells

In [6]:
tf.reduce_prod(tf.reduce_sum(one_n_j, axis =1)).numpy()

2566080.0

You can create your own samples by uncommenting or load ours.

In [7]:
# np.save("data/structural_zeros/SZ_NY_one_n_j.npy", one_n_j)

# X_ij_complete = np.array(df.values, dtype= int)

# # Shuffle the indices of the data
# indices = np.arange(X_ij_complete.shape[0])

# X_ij_complete_shuffled = X_ij_complete[indices,:]
# np.save("data/structural_zeros/SZ_NY_indices.npy", indices)
# np.save("data/structural_zeros/SZ_NY_X_ij_full.npy", X_ij_complete_shuffled)

# n = 1000
# X_ij_included = X_ij_complete_shuffled[:n,:]
# np.save("data/structural_zeros/SZ_NY_X_ij_1000.npy", X_ij_included)

# n = 5000
# X_ij_included = X_ij_complete_shuffled[:n,:]
# np.save("data/structural_zeros/SZ_NY_X_ij_5000.npy", X_ij_included)

# n = 10000
# X_ij_included = X_ij_complete_shuffled[:n,:]
# np.save("data/structural_zeros/SZ_NY_X_ij_10000.npy", X_ij_included)

In [8]:
os.chdir("scripts/")
%run -i mixed_membership_model.py
%run -i MVR.py
os.chdir("../")

Check the number of sample unique individuals

In [10]:
one_n_j = tf.convert_to_tensor(np.load("data/structural_zeros/SZ_NY_one_n_j.npy"), dtype = tf.int32)
X_ij    = tf.convert_to_tensor(np.load("data/structural_zeros/SZ_NY_X_ij_full.npy"), dtype = tf.int32)

N = 953076

batch_size = 2048

n = 1000
X_ij_1000 = np.load("data/structural_zeros/SZ_NY_X_ij_1000.npy")
tau_in_data_1000 = step_tau_with_data(X_ij_1000, one_n_j, X_ij[n:N,:], batch_size, N)
print(r"n=1000 with tau: ", tau_in_data_1000.numpy())

n = 5000
X_ij_5000 = np.load("data/structural_zeros/SZ_NY_X_ij_5000.npy")
tau_in_data_5000 = step_tau_with_data(X_ij_5000, one_n_j, X_ij[n:N,:], batch_size, N)
print(r"n=5000 with tau: ", tau_in_data_5000.numpy())

n = 10000
X_ij_10000 = np.load("data/structural_zeros/SZ_NY_X_ij_10000.npy")
tau_in_data_10000 = step_tau_with_data(X_ij_10000, one_n_j, X_ij[n:N,:], batch_size, N)
print(r"n=10000 with tau: ", tau_in_data_10000.numpy())


n=1000 with tau:  11.0
n=5000 with tau:  55.0
n=10000 with tau:  88.0


## Build the disjoint constraints

In [11]:
os.chdir("scripts/")
%run -i BNP_structural_zeros.py
%run -i manage_constraints.py
os.chdir("../")

In [12]:
current_constraint      = tf.convert_to_tensor([[1, 2, 0, 0, 0, 0,  0, 0, 0, 0],
						[1, 3, 0, 0, 0, 0,  0, 0, 0, 0],
						[1, 4, 0, 0, 0, 0,  0, 0, 0, 0],
						[2, 1, 0, 0, 0, 0,  0, 0, 0, 0],
						[3, 2, 0, 0, 0, 0,  0, 0, 0, 0],
						[3, 3, 0, 0, 0, 0,  0, 0, 0, 0],
						[3, 4, 0, 0, 0, 0,  0, 0, 0, 0],
						[0, 0, 1, 0, 1, 0,  0, 0, 0, 0],
						[0, 0, 1, 0, 2, 0,  0, 0, 0, 0],
						[0, 0, 1, 0, 3, 0,  0, 0, 0, 0],
						[0, 0, 1, 0, 4, 0,  0, 0, 0, 0],
						[0, 0, 1, 0, 5, 0,  0, 0, 0, 0],
						[0, 0, 1, 0, 0, 0,  8, 0, 0, 0],
						[0, 0, 1, 0, 0, 0,  9, 0, 0, 0],
						[0, 0, 1, 0, 0, 0, 10, 0, 0, 0],
						[0, 0, 1, 0, 0, 0, 11, 0, 0, 0],
						[0, 0, 2, 0, 0, 0,  8, 0, 0, 0],
						[0, 0, 2, 0, 0, 0,  9, 0, 0, 0],
						[0, 0, 2, 0, 0, 0, 10, 0, 0, 0],
						[0, 0, 2, 0, 0, 0, 11, 0, 0, 0],
						[0, 0, 3, 0, 0, 0, 10, 0, 0, 0],
						[0, 0, 3, 0, 0, 0, 11, 0, 0, 0],
						[0, 0, 4, 0, 0, 0, 10, 0, 0, 0],
						[0, 0, 4, 0, 0, 0, 11, 0, 0, 0],
						[0, 0, 1, 0, 0, 0,  0, 2, 0, 0],
						[0, 0, 1, 0, 0, 0,  0, 3, 0, 0],
						[0, 0, 1, 0, 0, 0,  0, 4, 0, 0],
						[0, 0, 2, 0, 0, 0,  0, 2, 0, 0],
						[0, 0, 2, 0, 0, 0,  0, 3, 0, 0],
						[0, 0, 2, 0, 0, 0,  0, 4, 0, 0],
						[0, 0, 3, 0, 0, 0,  0, 1, 0, 0],
						[0, 0, 4, 0, 0, 0,  0, 1, 0, 0],
						[0, 0, 5, 0, 0, 0,  0, 1, 0, 0],
						[0, 0, 6, 0, 0, 0,  0, 1, 0, 0],
						[0, 0, 7, 0, 0, 0,  0, 1, 0, 0],
						[0, 0, 8, 0, 0, 0,  0, 1, 0, 0],
						[0, 0, 9, 0, 0, 0,  0, 1, 0, 0],
						[0, 0, 1, 0, 0, 0,  0, 0, 2, 0],
						[0, 0, 1, 0, 0, 0,  0, 0, 3, 0],
						[0, 0, 2, 0, 0, 0,  0, 0, 2, 0],
						[0, 0, 2, 0, 0, 0,  0, 0, 3, 0],
						[0, 0, 3, 0, 0, 0,  0, 0, 1, 0],
						[0, 0, 4, 0, 0, 0,  0, 0, 1, 0],
						[0, 0, 5, 0, 0, 0,  0, 0, 1, 0],
						[0, 0, 6, 0, 0, 0,  0, 0, 1, 0],
						[0, 0, 7, 0, 0, 0,  0, 0, 1, 0],
						[0, 0, 8, 0, 0, 0,  0, 0, 1, 0],
						[0, 0, 9, 0, 0, 0,  0, 0, 1, 0],
						[0, 0, 1, 0, 0, 0,  0, 0, 0, 2],
						[0, 0, 1, 0, 0, 0,  0, 0, 0, 3],
						[0, 0, 2, 0, 0, 0,  0, 0, 0, 2],
						[0, 0, 2, 0, 0, 0,  0, 0, 0, 3],
						[0, 0, 3, 0, 0, 0,  0, 0, 0, 2],
						[0, 0, 3, 0, 0, 0,  0, 0, 0, 3],
						[0, 0, 4, 0, 0, 0,  0, 0, 0, 1],
						[0, 0, 5, 0, 0, 0,  0, 0, 0, 1],
						[0, 0, 6, 0, 0, 0,  0, 0, 0, 1],
						[0, 0, 7, 0, 0, 0,  0, 0, 0, 1],
						[0, 0, 8, 0, 0, 0,  0, 0, 0, 1],
						[0, 0, 9, 0, 0, 0,  0, 0, 0, 1],
						], dtype = tf.int32)
						

In [13]:
SZ_NY_one_n_j = tf.cast(np.load("data/structural_zeros/SZ_NY_one_n_j.npy"), dtype = tf.int32)

domain_j = tf.reduce_sum(SZ_NY_one_n_j, axis = 1)

Find the dijoint constraints (the code below can take more than 10min)

In [14]:
# disjoint_constraint = iteration_disjoint_constraint(domain_j, current_constraint)
# np.save("Data/NY/structural_zeros/SZ_NY_disjoint_constraint_MV.npy", disjoint_constraint)

In [15]:
disjoint_constraint = np.load("data/structural_zeros/SZ_NY_disjoint_constraint.npy")

Number of disjoint constraints

In [16]:
disjoint_constraint.shape[0]

557

which represents a total number of cells of:

In [17]:
ncells = tf.where((disjoint_constraint-1)==-1, tf.expand_dims(domain_j, axis = 0), tf.ones(tf.shape(disjoint_constraint), tf.int32))
tf.reduce_sum(tf.reduce_prod(ncells, axis = 1)).numpy()

2317030

# New York data: without structural zeros

Remove the structural zeros by considering AGE >18 and by removing OWNERSHIP and MORTGAGE.

In [26]:
df_no_zeros = df[['AGE', 'SEX', 'MARST', 'RACESING', 'EDUC', 'EMPSTAT', 'DISABWRK', 'VETSTAT']].copy()

In [27]:
print("AGE")
print(np.unique(df_no_zeros[["AGE"]], return_counts=True)[0])
print(np.unique(df_no_zeros[["AGE"]], return_counts=True)[1])

df_no_zeros = df_no_zeros.loc[df_no_zeros["AGE"]>3].copy()

print("AGE")
print(np.unique(df_no_zeros[["AGE"]], return_counts=True)[0])
print(np.unique(df_no_zeros[["AGE"]], return_counts=True)[1])

AGE
[0 1 2 3 4 5 6 7 8]
[201062  13292  13093  13455  82540 145106 219893 177662  86973]
AGE
[4 5 6 7 8]
[ 82540 145106 219893 177662  86973]


Size of the dataframe after removing individuals under 18.

In [28]:
len(df_no_zeros)

712174

In [29]:
for categorical in df_no_zeros.columns: 
	print(categorical)
	print(np.unique(df_no_zeros[[categorical]], return_counts=True)[0])
	print(np.unique(df_no_zeros[[categorical]], return_counts=True)[1])

	df_no_zeros[categorical]   = df_no_zeros[categorical] - df_no_zeros[categorical].min()
	
	print(categorical)
	print(np.unique(df_no_zeros[[categorical]], return_counts=True)[0])
	print(np.unique(df_no_zeros[[categorical]], return_counts=True)[1])

AGE
[4 5 6 7 8]
[ 82540 145106 219893 177662  86973]
AGE
[0 1 2 3 4]
[ 82540 145106 219893 177662  86973]
SEX
[0 1]
[335793 376381]
SEX
[0 1]
[335793 376381]
MARST
[0 1 2 3 4 5]
[361069  27218  22934  59345  55112 186496]
MARST
[0 1 2 3 4 5]
[361069  27218  22934  59345  55112 186496]
RACESING
[0 1 2 3 4]
[577672  91834   3657  36861   2150]
RACESING
[0 1 2 3 4]
[577672  91834   3657  36861   2150]
EDUC
[ 0  1  2  3  4  5  6  7  8  9 10]
[ 11119   4789  35184  17448  21938  24682 281471  89679  51338 101673
  72853]
EDUC
[ 0  1  2  3  4  5  6  7  8  9 10]
[ 11119   4789  35184  17448  21938  24682 281471  89679  51338 101673
  72853]
EMPSTAT
[1 2 3]
[413638  28626 269910]
EMPSTAT
[0 1 2]
[413638  28626 269910]
DISABWRK
[1 2]
[609000 103174]
DISABWRK
[0 1]
[609000 103174]
VETSTAT
[1 2]
[637522  74652]
VETSTAT
[0 1]
[637522  74652]


In [30]:
set_row = []
for row in df_no_zeros.columns:
	set_row.append(row)
	for col in df_no_zeros.columns:
		if row!=col:
			if (col in set_row)==False:
				tab = pd.crosstab( index=df_no_zeros[row], columns=df_no_zeros[col])
				print(tab.to_latex())

\begin{tabular}{lrr}
\toprule
SEX & 0 & 1 \\
AGE &  &  \\
\midrule
0 & 41736 & 40804 \\
1 & 70628 & 74478 \\
2 & 107208 & 112685 \\
3 & 83139 & 94523 \\
4 & 33082 & 53891 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrr}
\toprule
MARST & 0 & 1 & 2 & 3 & 4 & 5 \\
AGE &  &  &  &  &  &  \\
\midrule
0 & 6549 & 4314 & 791 & 413 & 225 & 70248 \\
1 & 66700 & 6321 & 4950 & 7622 & 460 & 59053 \\
2 & 138121 & 7255 & 9897 & 25340 & 2995 & 36285 \\
3 & 113935 & 4851 & 6019 & 21645 & 16338 & 14874 \\
4 & 35764 & 4477 & 1277 & 4325 & 35094 & 6036 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrr}
\toprule
RACESING & 0 & 1 & 2 & 3 & 4 \\
AGE &  &  &  &  &  \\
\midrule
0 & 62734 & 13436 & 564 & 5439 & 367 \\
1 & 112269 & 21430 & 914 & 9953 & 540 \\
2 & 177193 & 28669 & 1216 & 12078 & 737 \\
3 & 148166 & 20861 & 736 & 7517 & 382 \\
4 & 77310 & 7438 & 227 & 1874 & 124 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrrrrrr}
\toprule
EDUC & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 \\
AGE &  

In [33]:
os.chdir("scripts/")
%run -i mixed_membership_model.py
%run -i MVR.py
os.chdir("../")

In [34]:
n_j = np.zeros(len(df_no_zeros.columns))
for index in range(len(df_no_zeros.columns)):

    n_j[index] = df_no_zeros[df_no_zeros.columns[index]].max()
    
n_j = tf.convert_to_tensor(n_j+1, dtype = tf.float32)
max_categ = tf.expand_dims(tf.cast(tf.linspace(0, int(tf.reduce_max(n_j)) - 1, int(tf.reduce_max(n_j))), dtype = tf.float32), axis = 0)*tf.ones((tf.shape(n_j)[0], int(tf.reduce_max(n_j))))
ones_condition = max_categ<tf.expand_dims(n_j, axis = -1)*tf.ones((tf.shape(n_j)[0], int(tf.reduce_max(n_j))))

one_n_j = tf.where(ones_condition, tf.ones(tf.shape(ones_condition)), tf.zeros(tf.shape(ones_condition)))

The new contingency table has

In [35]:
tf.reduce_prod((tf.reduce_sum(one_n_j, axis = 1))).numpy()

39600.0

Again you can generate your data or load our own

In [36]:
# np.save("data/clean/NY_one_n_j.npy", one_n_j)

# X_ij_complete = np.array(df_no_zeros.values, dtype= int)

# # Shuffle the indices of the data
# indices = np.arange(X_ij_complete.shape[0])

# X_ij_complete_shuffled = X_ij_complete[indices,:]
# np.save("data/clean/NY_indices.npy", indices)
# np.save("data/clean/NY_X_ij_full.npy", X_ij_complete_shuffled)

# n = 1000
# X_ij_included = X_ij_complete_shuffled[:n,:]
# np.save("data/clean/NY_X_ij_1000.npy", X_ij_included)

# n = 5000
# X_ij_included = X_ij_complete_shuffled[:n,:]
# np.save("data/clean/NY_X_ij_5000.npy", X_ij_included)

# n = 10000
# X_ij_included = X_ij_complete_shuffled[:n,:]
# np.save("data/clean/NY_X_ij_10000.npy", X_ij_included)

Compute sample unique in this sample

In [37]:
one_n_j = tf.convert_to_tensor(np.load("data/clean/NY_one_n_j.npy"), dtype = tf.int32)
X_ij    = tf.convert_to_tensor(np.load("data/clean/NY_X_ij_full.npy"), dtype = tf.int32)

N = 712174

batch_size = 2048

n = 1000
X_ij_1000 = np.load("data/clean/NY_X_ij_1000.npy")
tau_in_data_1000 = step_tau_with_data(X_ij_1000, one_n_j, X_ij[n:N,:], batch_size, N)
print(r"n=1000 with tau: ", tau_in_data_1000.numpy())

n = 5000
X_ij_5000 = np.load("data/clean/NY_X_ij_5000.npy")
tau_in_data_5000 = step_tau_with_data(X_ij_5000, one_n_j, X_ij[n:N,:], batch_size, N)
print(r"n=5000 with tau: ", tau_in_data_5000.numpy())

n = 10000
X_ij_10000 = np.load("data/clean/NY_X_ij_10000.npy")
tau_in_data_10000 = step_tau_with_data(X_ij_10000, one_n_j, X_ij[n:N,:], batch_size, N)
print(r"n=10000 with tau: ", tau_in_data_10000.numpy())

n=1000 with tau:  9.0
n=5000 with tau:  27.0
n=10000 with tau:  53.0


# Synthetic data

In [40]:
os.chdir("scripts/")
%run -i mixed_membership_model.py
%run -i MVR.py
os.chdir("../")

Generate synthetic data as follows. Note that we did not set a seed, but we provide the synthetic data we generated.

In [41]:
one_n_j = tf.convert_to_tensor(np.load("data/clean/NY_one_n_j.npy"), dtype = tf.float32)
a_0, b_0 = 2, 1

K = 15

N = 712174

X_ij, theta_j_k, g_i, g_0 = param_mixed_membership_model(K, one_n_j, a_0, b_0, N)

# np.save("data/synthetic/synth_X_ij_full.npy", X_ij)
# np.save("data/synthetic/synth_theta_j_k.npy", theta_j_k)
# np.save("data/synthetic/synth_g_i.npy", g_i)
# np.save("data/synthetic/synth_g_0.npy", g_0)
# np.save("data/synthetic/US_one_n_j.npy", one_n_j)

# n = 1000
# np.save("Data/Synthetic/synth_X_ij_1000.npy", X_ij[:n,:])
# n = 5000
# np.save("Data/Synthetic/synth_X_ij_5000.npy", X_ij[:n,:])
# n = 10000
# np.save("Data/Synthetic/synth_X_ij_10000.npy", X_ij[:n,:])

In [42]:
X_ij      = tf.convert_to_tensor(np.load("data/synthetic/synth_X_ij_full.npy"), dtype = tf.int32)
theta_j_k = tf.convert_to_tensor(np.load("data/synthetic/synth_theta_j_k.npy"), dtype = tf.float32)
g_i       = tf.convert_to_tensor(np.load("data/synthetic/synth_g_i.npy" ), dtype = tf.float32)
g_0       = tf.convert_to_tensor(np.load("data/synthetic/synth_g_0.npy" ), dtype = tf.float32)
one_n_j   = tf.convert_to_tensor(np.load("data/synthetic/NY_one_n_j.npy"), dtype = tf.float32)

In [43]:
batch_size = 2048

X_ij_1000 = np.load("data/synthetic/synth_X_ij_1000.npy")
tau_in_data_1000 = step_tau_with_data(X_ij_1000, one_n_j, X_ij[n:N,:], batch_size, N)
print(r"n=1000 with tau: ", tau_in_data_1000.numpy())

X_ij_5000 = np.load("data/synthetic/synth_X_ij_5000.npy")
tau_in_data_5000 = step_tau_with_data(X_ij_5000, one_n_j, X_ij[n:N,:], batch_size, N)
print(r"n=5000 with tau: ", tau_in_data_5000.numpy())

X_ij_10000 = np.load("data/synthetic/synth_X_ij_10000.npy")
tau_in_data_10000 = step_tau_with_data(X_ij_10000, one_n_j, X_ij[n:N,:], batch_size, N)
print(r"n=10000 with tau: ", tau_in_data_10000.numpy())

n=1000 with tau:  4.0
n=5000 with tau:  28.0
n=10000 with tau:  66.0
